# 🗺️ Planners (with FunctionCallingStepwise planner)

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.18.0-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.18.0-preview"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    // .AddAzureOpenAIChatCompletion(
    //     deploymentName: Secrets.DeploymentName,
    //     endpoint: Secrets.Endpoint,
    //     apiKey: Secrets.ApyKey)
    .AddOpenAIChatCompletion(
        modelId: Secrets.OpenAIModel,
        apiKey: Secrets.OpenAIApiKey)    
    .Build();

### 🔌 Let's add a native plugin for use by the planner

In [ ]:
#!import ../../Helpers/PrintHelper.cs
#!import ../../Plugins/MotorPlugin.cs

kernel.Plugins.AddFromType<MotorPlugin>();

PrintHelper.PrintAllPluginFunctions(kernel);

### 📋 Prepare kernel arguments

In [ ]:
var goal = "You have a tree in front of the car. Avoid it.";
var kernelArguments = new KernelArguments()
{
    ["input"] = goal,
    ["commands"] = "go forward, go backward, turn left, turn right, and stop"
};

### Refine the goal

In [ ]:
var commandsPlugin = kernel.CreatePluginFromPromptDirectory(System.IO.Path.Combine(System.IO.Directory.GetCurrentDirectory(), "../../Plugins", "CommandsPlugin"), "CommandsPlugin");
kernel.Plugins.Add(commandsPlugin);
var semanticFunction = commandsPlugin["ExtractBasicCommands"];
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);
var refinedGoal = response.GetValue<string>();

kernelArguments["input"] = refinedGoal;

display(refinedGoal)

### 🗺️🧠 Generate FunctionCallingStepwise Plan

This planner is different from the handlebarsplanner in that it doesn't generate a plan ahead of time, and simply progresses towards its goal.

### 🏃 Let's see it run

In [ ]:
using Microsoft.SemanticKernel.Planning;

#pragma warning disable SKEXP0060

var functionCallingStepwisePlannerOptions = new FunctionCallingStepwisePlannerOptions
{
    MaxIterations = 30
};

functionCallingStepwisePlannerOptions.ExcludedPlugins.Add("CommandsPlugin");

PrintHelper.PrintAllPluginFunctions(kernel);

In [ ]:
#pragma warning disable SKEXP0060

var planner = new FunctionCallingStepwisePlanner(functionCallingStepwisePlannerOptions);
var plannerResult = await planner.ExecuteAsync(kernel, refinedGoal);

display(plannerResult.FinalAnswer)

In [ ]:
#pragma warning disable SKEXP0060

functionCallingStepwisePlannerOptions.ExcludedPlugins.Add("CommandsPlugin");
var planner = new FunctionCallingStepwisePlanner(functionCallingStepwisePlannerOptions);
var plannerResult = await planner.ExecuteAsync(kernel, refinedGoal);

display(plannerResult.FinalAnswer)